<a href="https://colab.research.google.com/github/afeld/python-public-policy/blob/master/lecture_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NYU Wagner - Python Coding for Public Policy**
# Class 6
- Review of for loops and functions
- Intro to APIs
- Intro to JSON data
- Into to web scraping (uses for loops!)

# LECTURE

In [1]:
import pandas as pd
import matplotlib.pyplot as plt 

In [2]:
# You can use pd.set_option() to make sure you see all the rows and columns in your dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

<ipython-input-2-457a47689858>:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


## Review of for loops

Important things to know:
- It will run for as many items as there are in the "iterable"
- The var be can used inside the for loop
- Identation is critical
- Values can be stored inside the for loop across the iterations

![alt text](https://www.learnbyexample.org/wp-content/uploads/python/Python-for-Loop-Syntax.png)

Breaking a for loop - how and why? https://www.tutorialspoint.com/python/python_break_statement.htm

## Review of functions

![alt text](https://www.askpython.com/wp-content/uploads/2019/06/python-functions.png)

## Intro to APIs

**API = Application Programming Interface**
- A set of routines, protocols, and tools for building software applications
- Specifies how software components should interact
- Transactions generally occur through a URL

**Things APIs are capable of doing:**

CRUD = CREATE, READ, UPDATE, DELETE

aka

POST, GET, PUT, and DELETE (equivalent HTTP methods)

They often require an "API key" which secures access to the API

**Twitter Trending Example**

Twitter docs: https://developer.twitter.com/en/docs/trends/trends-for-location/api-reference/get-trends-place

## Intro to JSON objects

JSON objects are usually what you get back from an API. They are a non-tabular way of storing data, and they are generally the preferred data format for web development. JSONs are essentially dictionaries.

In [3]:
import requests
request = requests.get('https://geosearch.planninglabs.nyc/v1/autocomplete?text=120 broad')

json_data = request.json()
print(json_data)

{'geocoding': {'version': '0.2', 'attribution': 'http://geosearch.planninglabs.nyc/attribution', 'query': {'text': '120 broad', 'parser': 'addressit', 'parsed_text': {}, 'tokens': ['120', 'broad'], 'size': 10, 'private': False, 'lang': {'name': 'English', 'iso6391': 'en', 'iso6393': 'eng', 'defaulted': True}}, 'engine': {'name': 'Pelias', 'author': 'Mapzen', 'version': '1.0'}, 'timestamp': 1587190277136}, 'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-74.01054, 40.708225]}, 'properties': {'id': '3903', 'gid': 'nycpad:address:3903', 'layer': 'address', 'source': 'nycpad', 'source_id': '3903', 'name': '120 BROADWAY', 'housenumber': '120', 'street': 'BROADWAY', 'postalcode': '10271', 'accuracy': 'point', 'country': 'United States', 'country_gid': 'whosonfirst:country:85633793', 'country_a': 'USA', 'region': 'New York State', 'region_gid': 'whosonfirst:region:0', 'region_a': 'NY', 'county': 'New York County', 'county_gid': 'whos

How to access an attribute in a json object using indexes

In [4]:
# get the coordinates
print(json_data['features'][0]['geometry']['coordinates'])

# get the BBL
json_data['features'][0]['properties']['pad_bbl']

# use numbers to index when there are multiple records in the dictionary

[-74.01054, 40.708225]


'1000477501'

## Intro to Web Scraping

In [5]:
from requests import get
from bs4 import BeautifulSoup

In [6]:
# Example scraping data from Give Directly live

raw_html = get('https://live.givedirectly.org/newsfeed/').content

html = BeautifulSoup(raw_html, 'html.parser')
print(html.prettify())

<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <title>
   GDLive | GiveDirectly
  </title>
  <meta content="authenticity_token" name="csrf-param">
   <meta content="bQ7iNKfDqt7nmvOoUeUHprb/8h3KGvL9EexyNH1iv4m+e5pXRNJncfgGKAMTHr+YFXR7lRogKZkj6B88/bzOFg==" name="csrf-token"/>
   <meta content="width=device-width, initial-scale=1" name="viewport"/>
   <!-- Favicons -->
   <link href="/assets/images/web_favicon_16px.png" rel="icon" type="image/icon"/>
   <link href="/assets/images/web_favicon_32px.png" rel="apple-touch-icon" sizes="32x32"/>
   <link href="/assets/images/web_favicon_48px.png" rel="apple-touch-icon" sizes="48x48"/>
   <link href="/assets/images/web_favicon_64px.png" rel="apple-touch-icon" sizes="64x64"/>
   <meta content="GiveDirectly: Send money directly to the extreme poor" property="og:title"/>
   <meta content="website" property="og:type"/>
   <meta content="https://s3-us-west-2.amazonaws.com/mobileappimages-prod/uploads/welcome-email-banner.png" property="og

In [7]:
df = pd.DataFrame(columns=['person', 'activity', 'quote'])

people = []
activities = []
quotes = []

# populate list of people
for i in (html.select('.card-name')):
  people.append(i.contents[0])
  print(people)

# populate list of activities
for i in (html.select('.card-phase-content')):
  activities.append(i.contents[2].replace('\n            ', '').replace('\n          ',''))

# populate list of quotes
for i in (html.select('.survey-answer-small')):
  quotes.append(i.contents[0].replace('\n ',''))

df.person = people
df.activity = activities
df.quote = quotes

df

['Mathew']
['Mathew', 'Wesley']
['Mathew', 'Wesley', 'Mwenda']
['Mathew', 'Wesley', 'Mwenda', 'Zawadi']
['Mathew', 'Wesley', 'Mwenda', 'Zawadi', 'Sidi']
['Mathew', 'Wesley', 'Mwenda', 'Zawadi', 'Sidi', 'Kabunda']
['Mathew', 'Wesley', 'Mwenda', 'Zawadi', 'Sidi', 'Kabunda', 'Kadzo']
['Mathew', 'Wesley', 'Mwenda', 'Zawadi', 'Sidi', 'Kabunda', 'Kadzo', 'Kadzo']
['Mathew', 'Wesley', 'Mwenda', 'Zawadi', 'Sidi', 'Kabunda', 'Kadzo', 'Kadzo', 'Jumwa']
['Mathew', 'Wesley', 'Mwenda', 'Zawadi', 'Sidi', 'Kabunda', 'Kadzo', 'Kadzo', 'Jumwa', 'Gladys']


,person,activity,quote
0,Mathew,enrolled.,"""Being a cobbler in an inaccessible premise cannot fetch me enough customers thus no much income to sustain my family of five. I spend all the money I get everyday to feed my family, leaving nothing for savings. My worry is that my children may drop out of school as this is not a reliable source of income yet I do not have more money to expand the business."""
1,Wesley,enrolled.,"""I have had to borrow my neighbor's mobile phone for this phone call enrollment because I do not have one. I will spend about $20 of my transfer to acquire a new mobile phone to ease communication and security of my transfer from GiveDirectly. Secondly, I want to add another new house in my homestead to make it two, so that my children do not have to go over to the neighborhood for sleep everyday. This jeopardizes their security especially my daughters. The house will cost approximately $300 inclusive of labour. \rSince I only rely on casual jobs in the village, I will purchase a dairy cow worth #300 to provide milk to supplement our usual meagre meals and sell the surplus for income."""
2,Mwenda,enrolled.,"""Owning a piece of land has always been my desire in my life. Currently, we are living in a 3-acre land which I share with my 4 brothers. I am planning to spend 700 dollars on purchase 2 acres of land once I receive my transfer. I will then use it for farming which will assure me of more harvest hence enough food for my family. Generally, this what receiving this money means to me."""
3,Zawadi,enrolled.,"""My house has collapsed 2 weeks ago due to the rains we are experiencing in our area. My wish has been to construct a good two-roomed irony sheet house but I have not fulfilled this due to financial constraints I am facing. Receiving this money means accomplishment of my desire since I am planning to spend 250 dollars on this. I will use the remaining amount to education my daughter who completed class 8 last year and I have failed to further her studies to secondary."""
4,Sidi,enrolled.,"""Supporting my daughter to do a dressmaking course so that she can be a reliable person in our family has been my greatest desire. Despite this greatest ambition, I have never managed to do this since I once took her to college but she was sent home after 3 days something that stressed me a lot but I could not do anything. My plan once I receive the cash grant is to use $200 to take her to the college. On the other hand, I intend to buy oxen and ox plow using $380 inclusively in order to help me upscaling my farming. This will mean a big boost to my family and hence, a better life."""
5,Kabunda,enrolled.,"""My desire has been to practice livestock keeping but due to financial challenges, I have not been able to accomplish it. Receiving this transfer means fulfilling it. I will use $ 90 to purchase 3 indigenous dairy cows. This will be along with term investment hence I will be able to sell at least 15 liters in a day and earn $ 45 which will be a good source of income for my family. I will use the remaining amount to pay school fees for the whole year for my child who is about to sit for her final exams. This will enable her to study comfortably she will not be sent home due to fees arrears. I am positive that she is going to pass with flying colors."""
6,Kadzo,enrolled.,"""Am an old woman who can no longer depend on myself. I rely on my children who are casual laborers and have their own families. Lack of a reliable source of income has been my biggest challenge to face."""
7,Kadzo,enrolled.,"""Poor health has become my biggest hardship in my life. This condition has made my life tough because am not able to do my casual jobs I used to do before,which could enable me earn some cash and sustain my needs. I depend on my grandchildren who is a herder."""
8,Jumwa,received a $501 second payment.,"""My life is different now because financial support empowered me to construct a decent house. Previously I was sheltering in 